In [1]:
import tensorflow as tf
# tf.debugging.enable_check_numerics() 
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import StringLookup

2025-06-06 13:25:27.051483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749216327.474325      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749216327.591320      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_images=np.load('/kaggle/input/complete-dataset/mini_processed_train_images.npy')
train_input_captions=np.load('/kaggle/input/complete-dataset/mini_processed_train_input_captions.npy')
train_label_captions=np.load('/kaggle/input/complete-dataset/mini_processed_train_label_captions.npy')

In [9]:
train_images[0].shape

(49, 2048)

In [3]:
train_images.shape

(5664, 49, 2048)

In [4]:
print(train_input_captions.shape,train_label_captions.shape)

(5664, 37) (5664, 37)


In [3]:
dev_images=np.load('/kaggle/input/complete-dataset/mini_processed_dev_images.npy')
dev_input_captions=np.load('/kaggle/input/complete-dataset/mini_processed_dev_input_captions.npy')
dev_label_captions=np.load('/kaggle/input/complete-dataset/mini_processed_dev_label_captions.npy')
print('loaded')

loaded


In [6]:
dev_images.shape

(1214, 49, 2048)

In [7]:
print(dev_input_captions.shape,dev_label_captions.shape)

(1214, 32) (1214, 32)


In [7]:
dataset=tf.data.Dataset.from_tensor_slices(((train_images,train_input_captions),train_label_captions)).batch(16)

I0000 00:00:1749216633.764393      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749216633.765118      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [4]:
emb_matrix=np.load('/kaggle/input/complete-dataset/new_embedding_matrix.npy')

In [5]:
emb_matrix.shape

(7828, 300)

In [6]:
# def masked_loss(y_true, y_pred):
#     # y_true: (batch_size, seq_len)
#     # y_pred: (batch_size, seq_len, vocab_size)
#     y_true = tf.cast(y_true, tf.int32)

#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
#     loss = loss_fn(y_true, y_pred)  # (batch_size, seq_len)
    
#     # Create mask for non-padding tokens
#     mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    
#     # Apply mask
#     loss *= mask

#     # Return average loss per batch
#     return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def masked_accuracy(y_true, y_pred):
    # y_true: (batch_size, seq_len)
    # y_pred: (batch_size, seq_len, vocab_size)
    y_true = tf.cast(y_true, tf.int32)

    y_pred_classes = tf.argmax(y_pred, axis=-1, output_type=tf.int32)  # (batch_size, seq_len)
    matches = tf.cast(tf.equal(y_true, y_pred_classes), tf.float32)

    # Mask out padding tokens
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    matches *= mask
    epsilon = 1e-7
    return tf.reduce_sum(matches) / tf.reduce_sum(mask)

In [51]:
def model_build():
    # strategy = tf.distribute.MirroredStrategy()
    # print(f"Building model with MirroredStrategy, num_replicas_in_sync: {strategy.num_replicas_in_sync}")
    
    #with strategy.scope():
        image = tf.keras.Input((49, 2048), name="image_input")
        input_caption = tf.keras.Input((None,), name="caption_input")

        x_image = Dense(64, activation='relu')(image)
        x_image = Dense(512, activation='relu')(x_image) # Shape: (batch, 49, 512)

        embedding_layer = Embedding(7828, 300, trainable=False, mask_zero=True) # mask_zero=False is fine as loss handles padding
        embedding_layer.build((None,))
        embedding_layer.set_weights([emb_matrix])

        x_caption = embedding_layer(input_caption) # Shape: (batch, seq_len, 300)
        x_caption = LSTM(512, return_sequences=True)(x_caption) # Shape: (batch, seq_len, 512)

        # Changed num_heads and key_dim for more standard MHA
        # num_heads * key_dim should ideally match the feature dimension (512 here for query)
        attention = MultiHeadAttention(num_heads=1, key_dim=512)(query=x_caption, value=x_image) # Output Shape: (batch, seq_len, 512)

        x = tf.keras.layers.Add()([x_caption, attention])
        x = LayerNormalization()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        #x = LSTM(448,return_sequences=True)(x)
        x = LSTM(448, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(0.3)(x)

        # Changed to linear activation for numerical stability with from_logits=True in loss
        logits = Dense(7828, activation=None, name="logits_layer")(x) # Raw logits
        

        model = tf.keras.Model(inputs=[image, input_caption], outputs=logits)
        
        

        model.compile(optimizer=Adam(learning_rate=0.0003), # REDUCED LR
              loss=SparseCategoricalCrossentropy(from_logits=True, ignore_class=0),
              metrics=[masked_accuracy])

        
        
        return model

In [52]:
model=model_build()

In [53]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ caption_input             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image_input (InputLayer)  │ (None, 49, 2048)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, None, 300)      │      2,348,400 │ caption_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_3 (NotEqual)    │ (None, None)           │              0 │ caption_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 49, 64)         │        131,136 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_4 (LSTM)             │ (None, None, 512)      │      1,665,024 │ embedding_2[0][0],     │
│                           │                        │                │ not_equal_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 49, 512)        │         33,280 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, None, 512)      │      1,050,624 │ lstm_4[0][0],          │
│ (MultiHeadAttention)      │                        │                │ not_equal_3[0][0],     │
│                           │                        │                │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, None, 512)      │              0 │ lstm_4[0][0],          │
│                           │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, None, 512)      │          1,024 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, None, 512)      │              0 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ logical_or_3 (LogicalOr)  │ (None, None)           │              0 │ not_equal_3[0][0],     │
│                           │                        │                │ not_equal_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, None, 448)      │      1,722,112 │ dropout_8[0][0],       │
│                           │                        │                │ logical_or_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, None, 448)      │              0 │ lstm_5[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ logits_layer (Dense)      │ (None, None, 7828)     │      3,514,772 │ dropout_9[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,466,372 (39.93 MB)

 Trainable params: 8,117,972 (30.97 MB)

 Non-trainable params: 2,348,400 (8.96 MB)

In [35]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',       # Save path
    monitor='val_loss',             # Metric to monitor
    save_best_only=True,            # Only save when val_loss improves
    save_weights_only=False,        # Save the entire model (set True for only weights)
    mode='min',                     # 'min' because we want to minimize val_loss
    verbose=1                       # Prints a message when saving
)


In [54]:
# Optional: prefetch for performance
train_dataset = dataset.prefetch(tf.data.AUTOTUNE)

print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=50, # Experimenting with: 1. Keep epochs moderate for testing the fix: 50
    validation_data=(([dev_images, dev_input_captions], dev_label_captions))
    #callbacks=[checkpoint_callback]
)

Starting training...
Epoch 1/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - loss: 5.7651 - masked_accuracy: 0.1858 - val_loss: 4.4163 - val_masked_accuracy: 0.2866
Epoch 2/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - loss: 4.0995 - masked_accuracy: 0.3103 - val_loss: 3.9864 - val_masked_accuracy: 0.3254
Epoch 3/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 3.6955 - masked_accuracy: 0.3457 - val_loss: 3.8004 - val_masked_accuracy: 0.3433
Epoch 4/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 3.4624 - masked_accuracy: 0.3621 - val_loss: 3.6831 - val_masked_accuracy: 0.3550
Epoch 5/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 3.2852 - masked_accuracy: 0.3752 - val_loss: 3.6124 - val_masked_accuracy: 0.3593
Epoch 6/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 3.1339 - masked_accuracy: 0.3891 - val_loss: 3.5631 - val_masked_accuracy: 0.3696
Epoch 7/50
354/354 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 3.0075 - masked_accuracy: 0.4010 - val_loss: 3.5270 -

In [38]:
model=tf.keras.models.load_model('/kaggle/working/best_model.keras',compile=False)

In [55]:
output=model.predict([train_images[:4],train_input_captions[:4]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step


In [40]:
output.shape

(2, 37, 7828)

In [56]:
predicted_indices = np.argmax(output, axis=-1)

In [42]:
predicted_indices.shape

(2, 37)

In [43]:
print(predicted_indices[0])
print(train_label_captions[0])

[  51 4034 3457   51 5446 5975 3541 6511   51   51 5594 4519 6504    3
   51 3476    3    3    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1]
[  51 1301 3457   51 4957 2121 3541 1398 7352   51 5892 4519 6504 3457
  218 2332 7565    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]


In [18]:
def load_from_disk(path):
    import pickle
    with open(path, "rb") as f:
        loaded_list = pickle.load(f)

    return loaded_list

In [19]:
vocab=load_from_disk('/kaggle/input/vocab-data/new_vocab.pkl')

In [20]:
vocab

['<SOS>',
 '<EOS>',
 '08',
 '1',
 '10',
 '104',
 '11',
 '12',
 '13',
 '157',
 '17',
 '19',
 '1915',
 '1923',
 '1950s',
 '2',
 '2008',
 '21',
 '22',
 '23',
 '25',
 '28',
 '281',
 '29',
 '3',
 '30',
 '32',
 '33',
 '3rd',
 '4',
 '42',
 '4x4',
 '5',
 '50',
 '52',
 '528',
 '57',
 '58',
 '6',
 '625',
 '661',
 '7',
 '75',
 '8',
 '80',
 '84',
 '90',
 '93',
 '97',
 'a',
 'abandon',
 'abandoned',
 'abdomen',
 'ability',
 'aboard',
 'aboriginal',
 'about',
 'above',
 'abs',
 'abseiling',
 'ac',
 'accelerates',
 'accent',
 'accented',
 'accents',
 'accepting',
 'accepts',
 'accessories',
 'accident',
 'accompanied',
 'accompanies',
 'accompanying',
 'accordion',
 'accross',
 'ace',
 'acoustic',
 'acrobat',
 'acrobatic',
 'acrobatics',
 'across',
 'act',
 'acting',
 'action',
 'actions',
 'active',
 'activities',
 'activity',
 'actor',
 'actors',
 'acts',
 'ad',
 'add',
 'additional',
 'addressing',
 'adhd',
 'adjust',
 'adjusting',
 'adjustments',
 'adjusts',
 'admire',
 'admired',
 'admires',
 'a

In [21]:
invert_vectorize=StringLookup(mask_token='',vocabulary=vocab,invert=True)

In [22]:
invert_vectorize.get_vocabulary()

['',
 '[UNK]',
 '<SOS>',
 '<EOS>',
 '08',
 '1',
 '10',
 '104',
 '11',
 '12',
 '13',
 '157',
 '17',
 '19',
 '1915',
 '1923',
 '1950s',
 '2',
 '2008',
 '21',
 '22',
 '23',
 '25',
 '28',
 '281',
 '29',
 '3',
 '30',
 '32',
 '33',
 '3rd',
 '4',
 '42',
 '4x4',
 '5',
 '50',
 '52',
 '528',
 '57',
 '58',
 '6',
 '625',
 '661',
 '7',
 '75',
 '8',
 '80',
 '84',
 '90',
 '93',
 '97',
 'a',
 'abandon',
 'abandoned',
 'abdomen',
 'ability',
 'aboard',
 'aboriginal',
 'about',
 'above',
 'abs',
 'abseiling',
 'ac',
 'accelerates',
 'accent',
 'accented',
 'accents',
 'accepting',
 'accepts',
 'accessories',
 'accident',
 'accompanied',
 'accompanies',
 'accompanying',
 'accordion',
 'accross',
 'ace',
 'acoustic',
 'acrobat',
 'acrobatic',
 'acrobatics',
 'across',
 'act',
 'acting',
 'action',
 'actions',
 'active',
 'activities',
 'activity',
 'actor',
 'actors',
 'acts',
 'ad',
 'add',
 'additional',
 'addressing',
 'adhd',
 'adjust',
 'adjusting',
 'adjustments',
 'adjusts',
 'admire',
 'admired',


In [58]:
print(invert_vectorize(predicted_indices[3]))
print(invert_vectorize(train_label_captions[3]))

tf.Tensor(
[b'a' b'man' b'and' b'on' b'a' b'bench' b'while' b'his' b'dog' b'sits'
 b'by' b'him' b'<EOS>' b'item' b'item' b'item' b'item' b'item' b'item'
 b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item'
 b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item'], shape=(37,), dtype=string)
tf.Tensor(
[b'a' b'man' b'lays' b'on' b'a' b'bench' b'while' b'his' b'dog' b'sits'
 b'by' b'him' b'<EOS>' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''], shape=(37,), dtype=string)


In [24]:
print(invert_vectorize(predicted_indices[1]))
print(invert_vectorize(train_label_captions[1]))

tf.Tensor(
[b'a' b'black' b'dog' b'and' b'a' b'black' b'dog' b'are' b'fighting'
 b'<EOS>' b'potato' b'potato' b'potato' b'potato' b'potato' b'potato'
 b'potato' b'potato' b'potato' b'potato' b'potato' b'potato' b'potato'
 b'potato' b'potato' b'potato' b'potato' b'potato' b'potato' b'potato'
 b'potato' b'potato' b'potato' b'potato' b'potato' b'potato' b'potato'], shape=(37,), dtype=string)
tf.Tensor(
[b'a' b'black' b'dog' b'and' b'a' b'spotted' b'dog' b'are' b'fighting'
 b'<EOS>' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''], shape=(37,), dtype=string)


In [59]:
output_dev=model.predict([dev_images[0:4],dev_input_captions[0:4]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step


In [60]:
predicted_indices_dev = np.argmax(output_dev, axis=-1)

In [64]:
print(invert_vectorize(predicted_indices_dev[0]))
print(invert_vectorize(dev_label_captions[0]))

tf.Tensor(
[b'a' b'group' b'of' b'[UNK]' b'is' b'<EOS>' b'a' b'[UNK]' b'<EOS>'
 b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item'
 b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item' b'item'
 b'item' b'item' b'item' b'item' b'item'], shape=(32,), dtype=string)
tf.Tensor(
[b'a' b'group' b'of' b'cyclers' b'race' b'around' b'a' b'track' b'<EOS>'
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b''], shape=(32,), dtype=string)


In [63]:
model.save('/kaggle/working/model.keras')